# This code prepares the multi-class image data

In [1]:
import numpy as np
import os
import skimage.io as io
import skimage.transform as trans

In [2]:
def prepare_multi_class_GT(GT_PATH, class_names, savepath, target_size=(512,512), n_class=3):

    f_names = os.listdir(GT_PATH+class_names[0])
    for files in f_names:
        GT_im=np.zeros(np.concatenate((target_size,n_class),axis=None)) #This creates a zero array of size (512,512,3)
        FG=np.zeros(target_size)
        
        for idx,cn in enumerate(class_names):
            lab=io.imread(GT_PATH+cn+files, as_gray=True)
            lab = trans.resize(lab,target_size)
            if(np.max(lab)>1):
                lab=lab/255
            lab[lab>=0.1]=1 #threshold at 0.1. Change this value based on your requirement
            lab[lab<0.1]=0
            
            if (idx<2): #Bright Lesions
                GT_im[:,:,1]=GT_im[:,:,1]+lab
            else:#Red Lesions
                GT_im[:,:,0]=GT_im[:,:,0]+lab
        if(np.sum(GT_im[:,:,0])>0):
            GT_im[:,:,0]=GT_im[:,:,0]/np.ptp(GT_im[:,:,0])
        if(np.sum(GT_im[:,:,1])>0):
            GT_im[:,:,1]=GT_im[:,:,1]/np.ptp(GT_im[:,:,1])
        FG=(GT_im[:,:,0]+GT_im[:,:,1]>0).astype(int)
        GT_im[:,:,2]=1-FG
        io.imsave(savepath+files,GT_im)
       

In [3]:
GT_PATH='./ddb1_groundtruth/'
class_names=['hemorrhages/','redsmalldots/','hardexudates/', 'softexudates/',]

prepare_multi_class_GT(GT_PATH,class_names, './GT/')

<ipython-input-2-b1b96e831623>:26: UserWarning: ./GT/image089.png is a low contrast image
  io.imsave(savepath+files,GT_im)
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-2-b1b96e831623>:26: UserWarning: ./GT/image088.png is a low contrast image
  io.imsave(savepath+files,GT_im)
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-2-b1b96e831623>:26: UserWarning: ./GT/image071.png is a low contrast image
  io.imsave(savepath+files,GT_im)
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-2-b1b96e831623>:26: UserWarning: ./GT/image032.png is a low contrast image
  io.imsave(savepath+files,GT_im)
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-2-b1b96e831623>:26: UserWarning: ./GT/image074.png is a low contrast image
  io.imsave(savepath+files,GT_im)
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-2-b1b96e831623>:26

# Now your data is ready. Partition it to train and test..

In [ ]:
RL = [255,0,0]
BL = [0,255,0]
Unlabelled = [0,0,0]
COLOR_DICT = np.array([ RL,BL, Unlabelled])
class_name = [ 'RL','BL', 'None']  # You must define by yourself


def labelVisualize(num_class,color_dict,img):
    img = img[:,:,0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i] = color_dict[i]
      
    return img_out


def saveResult(img_path,save_path,npyfile,flag_multi_class = True,num_class = 2):
    files=os.listdir(img_path)
    #print(len(img_path))
    #print(len(npyfile))
    
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        #img1=np.array(((img - np.min(img))/np.ptp(img))>0.6).astype(float)
        img[img>0.1]=1
        img[img<=0.1]=0
        io.imsave(os.path.join(save_path, files[i]+'_predict.png'),img)